# Start here

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import sys
from tqdm import tqdm
import pyvisa as visa
import time as py_time
import serial #pyserial
from serial.tools import list_ports
import os.path
from pathlib import Path #? do i use this even?
from datetime import datetime
import time
import ipywidgets as widgets
from ipywidgets import HBox, VBox
import os
from ctypes import *
# import ctypes
# import usbtmc

### Connect to arduino

You
I had to unplug and plug back in the USB cable for this to work

If you get the error

`SerialException: could not open port 'COM3': PermissionError(13, 'Access is denied.', None, 5)`

then the arduino is probably connected to somethign in a different notebook. Try killing/restarting the kernels in all other notebooks, or take the aggresive strategy of unplugging and replugging the arduino usb.


In [ ]:
ser = serial.Serial()
ser.baudrate= 9600
ser.port = 'COM3'
ser.open()
if ser.is_open:
    print("Successfully connected")


def pulse_arduino(ms: int = 1):
    """
    Send one pulse from the arduino

    Parameters
    ----------
    ms : int, default 1
        The number of the ms to have the arduino pulse last
    """
    ser.write(f"pulse_{int(ms)}".encode())

Successfully connected


### Connect to the oscilloscope
The output should look something like ```['USB0::0x1AB1::0x0588::DS1K00005888::INSTR']``` if successful.


Blog post about programming it: https://www.righto.com/2013/07/rigol-oscilloscope-hacks-with-python.html

Scope programming guide: https://www.rigolna.com/products/digital-oscilloscopes/ds1000e/ds1052e/#tabs-2

timebase={
    '2ns':'.000000002',
    '5ns':'.000000005',
    '10ns':'.00000001',
    '20ns':'.00000002',
    '50ns':'.00000005',
    '100ns':'.0000001',
    '200ns':'.0000002',
    '500ns':'.0000005',
    '900ns':'.0000009',
    '1us':'.000001',
    '2us':'.000002',
    '5us':'.000005',
    '10us':'.00001',
    '20us':'.00002',
    '50us':'.00005',
    '100us':'.0001'
}
time_dropdown = widgets.Dropdown(options = timebase)
time_dropdown

### Oscilloscope object and control


In [ ]:
class scope:

    timebase = {('2ns', '.000000002'), ('5ns', '.000000005'),
             ('10ns','.00000001'), ('20ns', '.00000002'),  ('50ns', '.00000005'),
             ('100ns','.0000001'), ('200ns', '.0000002'),  ('500ns', '.0000005'),
             ('1µs', '.000001'), ('2µs', '.000002'), ('5µs', '.000005'),
             ('10µs', '.00001'), ('20µs', '.00002'), ('50µs', '.00005'),
             ('100µs', '.0001'), ('200µs', '.0002'), ('500µs', '.0005'),
             ('1ms', '.001'), ('2ms', '.002'),  ('5ms', '.005'),
             ('10ms', '.01'), ('20ms', '.02'),  ('50ms', '.05'),
             ('100ms', '.1'), ('200ms', '.2'),  ('500ms', '.5'),
            ('1s', '1'), ('2s', '2'),  ('5s', '5'),
            ('10s', '10'), ('20s', '20'),  ('50s', '50')}

    modes = {('Edge', 'EDGE'),('Pulse' , 'PULS'), ('Slope', 'SLOP'),
             ('Video','VIDEO'),('Pattern', 'PATT'),('Duration', 'DUR'),
             ('Alteration', 'ALT')}

    modes_sweep = {'EDGE': ['AUTO', 'NORM', 'SING'], 'PULS': ['AUTO', 'NORM', 'SING'],
             'SLOP': ['AUTO', 'NORM', 'SING'], 'VIDEO':[],
             'PATT': ['AUTO', 'NORM', 'SING'], 'DUR': ['AUTO', 'NORM', 'SING'],
             'ALT': ['AUTO', 'NORM', 'SING']}

    def __init__(self):
        # load the scope whatever
        rm = visa.ResourceManager()
        insts = rm.list_resources()
        usb = list(filter(lambda x: 'USB' in x, insts))
        self._scope = rm.open_resource(usb[0], timeout=20000, chunk_size=1024000)

#         set the initial values
        wait_time = .1
        time.sleep(wait_time)
        self._chan1_voff = float(self._scope.query_ascii_values(":CHAN1:OFFS?")[0])
        self._chan2_voff = float(self._scope.query_ascii_values(":CHAN2:OFFS?")[0])
        self.set_vscale(1, 1)
        self.set_trigger(mode = 'EDGE', source = 'EXT')
        time.sleep(wait_time)
        self.set_sweep('SING')
        self.set_wav('RAW')
        self._trigger_range = 1.2
        self.set_triggerlvl(0.2)
        self._toff = float(self._scope.query_ascii_values(':TIM:OFFS?')[0])
        time.sleep(wait_time)
        self._ts = float(self._scope.query_ascii_values(':TIM:SCAL?')[0])
        time.sleep(wait_time)
        print('done')

#         DEFINITONS
    def run(self):
        self._scope.write(":RUN")

    def stop(self):
        self._scope.write(":STOP")

    def set_vscale(self, chan1=None, chan2=None):
        if chan1 is not None:
            self._scope.write(":CHAN1:SCAL "+ str(chan1))
            time.sleep(0.1)
            self._chan1_scale = float(self._scope.query_ascii_values(':CHAN1:SCAL?')[0])
            self.set_voff(chan1 = self._chan1_voff, chan2 = None)
        if chan2 is not None:
            self._scope.write(":CHAN2:SCAL "+ str(chan2))
            time.sleep(0.1)
            self._chan2_scale = float(self._scope.query_ascii_values(':CHAN2:SCAL?')[0])
            self.set_voff(chan1 = None, chan2 = self._chan2_voff)

    def set_voff(self, chan1=None, chan2=None):
        if chan1 is not None:
            self._scope.write(":CHAN1:OFFS " + str(chan1))
            time.sleep(0.1)
            self._chan1_voff = float(self._scope.query_ascii_values(':CHAN1:OFFS?')[0])
        if chan2 is not None:
            self._scope.write(":CHAN2:OFFS "+ str(chan2))
            time.sleep(0.1)
            self._chan2_voff = float(self._scope.query_ascii_values(':CHAN2:OFFS?')[0])

    def set_trigger(self, mode = None, source = None):
        ##Set mode
        if mode is not None:
            self._scope.write(":TRIG:MODE "+ mode) #make sure the options offered by the widget have the correct values
            self._trigger_mode = mode
        ##Set source
        if source is not None:
            self._scope.write(":TRIG:"+self._trigger_mode +":SOUR "+ source)
            self._trigger_source = source
        ##Adjust scale to source
        if self._trigger_source == 'CHAN1':
            scale = self._chan1_scale #what is scale useful for?
        elif (self._trigger_source == 'CHAN2'):
            scale = self._chan2_scale
        else:
            if self._chan1_scale > self._chan2_scale:
                scale = self._chan1_scale
            else:
                scale = self._chan2_scale
        ##Ajdust range
        if  self._trigger_mode == 'EXT':
            self._trigger_range=1.2
        else:
            self._trigger_range=6*scale


    def set_triggerlvl(self, level):
        ## Set level
        if (level >= (- self._trigger_range) and level <= self._trigger_range):
            self._scope.write(":TRIG:"+ self._trigger_mode+":LEV "+str(level))
            # this causes a timeout
            # self._triggerlvl = float(self._scope.query_ascii_values(':TRIG:'+ self._trigger_mode+':LVL?')[0])
            self._triggerlvl = level
        else:
            self.stop()
            #i forgot how to do error catches but this is okay for now
            print("level out of range, range is: +/-" + str(self._trigger_range))

    def set_sweep(self, sweep):
        self._scope.write(':TRIG:'+ self._trigger_mode+':SWE ' + sweep)
        self._sweep = sweep

    def set_wav(self, wav):
        self._scope.write(':WAV:POIN:MODE '+ wav)

    def set_timescale(self, timescale=None, offset=None):
        if timescale is not None:
            self._scope.write(':TIM:SCAL '+str(timescale))
            time.sleep(1)
            self._ts = float(self._scope.query_ascii_values(':TIM:SCAL?')[0])
        if offset is not None:
            self._scope.write(':TIM:OFFS '+str(offset))
            time.sleep(1)
            self._toff = float(self._scope.query_ascii_values(':TIM:OFFS?')[0])

    def get_data(self, channel):
        self._scope.write(":RUN")
#       shouldn't be neccesary but let's see if it helps:
        self._scope.write(":WAV:POIN:MODE RAW")
        if channel == "CHAN1":
            self._scope.write(":WAV:DATA? CHAN1")
#             rawdata = self._scope.query(":WAV:DATA? CHAN1").encode('ascii')[10:]

        elif channel == "CHAN2":
#             rawdata = self._scope.query(":WAV:DATA? CHAN2").encode('ascii')[10:]
            self._scope.write(":WAV:DATA? CHAN2")
        time.sleep(0.5)
        rawdata = self._scope.read_raw()[10:]
        self._scope.write(":STOP")
        return np.frombuffer(rawdata,'B')

    def write(self, text):
        self._scope.write(text)

scope_cntl = scope()


done


# Functions for oscilloscope

In [ ]:
def collect(delay=4, chan1 = True, chan2 = True):

    scope_cntl.run()
    pulse_arduino()
    time.sleep(delay)
    if chan1 and chan2:
        data1 = scope_cntl.get_data("CHAN1")
        print('data 1 size ' + str(data1.size))
        data2 = scope_cntl.get_data("CHAN2")
        print('data 2 size ' + str(data2.size))
        return data1, data2
    if chan1:
        data1 = scope_cntl.get_data("CHAN1")
        print('data 1 size ' + str(data1.size))
        return data1
    if chan2:
        data2 = scope_cntl.get_data("CHAN2")
        print('data 2 size ' + str(data2.size))
        return data2

#     return np.vstack([data1[None, :], data2[None, :]]), data1, data2

Scales accoridng to: https://www.righto.com/2013/07/rigol-oscilloscope-hacks-with-python.html


In [ ]:
def make_name(name):
    folder = "C:\\Users\\LabUser\\Documents\\Stark_Effect\\spectroscopy-graphene-device\\Pulseoutput\\" + datetime.today().strftime('%y-%m-%d')

    if not os.path.isdir(folder):
        os.makedirs(folder)

    fn = folder +"\\" + datetime.today().strftime('%y-%m-%d') + str(name)
    i = 1

    while (os.path.exists(fn + '_' + str(i) + '.png') or os.path.exists(fn + '_' + str(i)+ '.npz')):
        i += 1

    fn = fn + '_' + str(i)
    return fn


In [ ]:
'''
Create a plot and save in in a unique place with date and voltage in name
'''

'''
the v scale is fixed to 0 to 150 for the current experiment but it needs ot be adjusted
'''

def saver(data1 = None, data2 = None, name = "Untitled"):

    fn = make_name(name)

    if data1 is not None and data2 is not None:
        if data1.size == data2.size:
#             t1= t2 = np.arange(-int(data1.size/2),int(data1.size/2))*scope_cntl._ts/50+scope_cntl._toff
            t1 = t2 = np.linspace(scope_cntl._toff - 6 * scope_cntl._ts, scope_cntl._toff + 6 * scope_cntl._ts, num=len(data1))
        else:
#             t1 = np.arange(-int(data1.size/2),int(data1.size/2))*scope_cntl._ts/50+scope_cntl._toff
#             t2 = np.arange(-int(data2.size/2),int(data2.size/2))*scope_cntl._ts/50+scope_cntl._toff
# see if that makes sense:
            t1 = np.linspace(scope_cntl._toff - 6 * scope_cntl._ts, scope_cntl._toff + 6 * scope_cntl._ts, num=len(data1))
            t2 = np.linspace(scope_cntl._toff - 6 * scope_cntl._ts, scope_cntl._toff + 6 * scope_cntl._ts, num=len(data2))


        np.savez(
            fn,
            time1 = t1,
            time2 = t2,
            timeoffset=np.array([scope_cntl._toff]),
            timescale=np.array([scope_cntl._ts]),
            voltscale = np.array([scope_cntl._chan1_scale, scope_cntl._chan2_scale],dtype=float),
            voltoffset = np.array([scope_cntl._chan1_voff, scope_cntl._chan2_voff],dtype=float),
            raw_data_chan1 = data1,
            raw_data_chan2 = data2
        )

#         data1_avg = moving_average(data1, 20)
        #create plot
        fig, ax1 = plt.subplots()
        ax1.plot(t1, data1, label = "EF", color = 'blue')
        ax1.set_xlabel('time (s)')
        ax2 = ax1.twinx()
        ax2.plot(t2, data2, label = "Photodiode", color = 'red')
        plt.title('Measurements '+name)
        ax1.legend()
        ax2.legend()
        ax1.set_ylabel('voltage (V)')
#  for now:
        ax2.set_ylim([0, 150])
        ax2.set_ylabel('Intensity(au)')

    elif data1 is not None:
        t = np.arange(-int(data1.size/2),int(data1.size/2))*scope_cntl._ts/50+scope_cntl._toff
        np.savez(
            fn,
            time = t,
#             chan1= volts[0],
            timeoffset=np.array([scope_cntl._toff]),
            timescale=np.array([scope_cntl._ts]),
            voltscale = np.array([scope_cntl._chan1_scale,scope_cntl._chan2_scale],dtype=float),
            voltoffset = np.array([scope_cntl._chan1_voff, scope_cntl._chan2_voff],dtype=float),
            raw_data_chan1 = data1
        )
        #create plot
        fig, ax1 = plt.subplots()
        ax1.plot(t, data1, label = "EF")
        ax1.set_xlabel('time (s)')
        plt.title('Measurements '+name)
        #its ugly this way but what can you do
        ax1.legend()
        ax1.set_ylabel('voltage (V)')

    elif data2 is not None:
        t = np.arange(-int(data2.size/2),int(data2.size/2))*scope_cntl._ts/50+scope_cntl._toff
        np.savez(
            file = fn,
            time= t,
            timeoffset=np.array([scope_cntl._toff]),
            timescale=np.array([scope_cntl._ts]),
            voltscale = np.array([scope_cntl._chan1_scale, scope_cntl._chan2_scale],dtype=float),
            voltoffset = np.array([scope_cntl._chan1_voff, scope_cntl._chan2_voff],dtype=float),
            raw_data_chan2 = data2
        )

        #create plot
        fig, ax2 = plt.subplots()
        ax2.set_xlabel('time (s)')
        ax2.plot(t, data2, label = "Photodiode", color = 'red')
        plt.title('Measurements '+name)
        #its ugly this way but what can you do
        ax2.legend()
        #  for now:
        ax2.set_ylim([0, 150])
        ax2.set_ylabel('Intensity(au)')
    else:
        print('What do you want?')
        return 0

    fig.savefig(fn + '.png')
    fig.show()

# Polarisers

In [ ]:
if sys.version_info < (3, 8):
    os.chdir(r"C:\Program Files\Thorlabs\Kinesis")
else:
    os.add_dll_directory(r"C:\Program Files\Thorlabs\Kinesis")

lib: CDLL = cdll.LoadLibrary("C:\Program Files\Thorlabs\Kinesis\Thorlabs.MotionControl.KCube.DCServo.dll")

# Set up the device to convert real units to device units
STEPS_PER_REV = c_double(1919.64186)  # for the PRM1-Z8
gbox_ratio = c_double(1.0)  # gearbox ratio
pitch = c_double(1.0)

In [ ]:
class polarizer:
    STEPS_PER_REV = c_double(1919.64186)  # for the PRM1-Z8
    gbox_ratio = c_double(1.0)  # gearbox ratio
    pitch = c_double(1.0)

    def __init__(self):
#         if sys.version_info < (3, 8):
#             os.chdir(r"C:\Program Files\Thorlabs\Kinesis")
#         else:
#             os.add_dll_directory(r"C:\Program Files\Thorlabs\Kinesis")
#         lib: CDLL = cdll.LoadLibrary("C:\Program Files\Thorlabs\Kinesis\Thorlabs.MotionControl.KCube.DCServo.dll")

        #set parameter
        self.set_position('front')
        self.home()
        self.set_position('back')
        self.home()

    def set_position(self, pol): #do i need ot refernece self??
        if lib.TLI_BuildDeviceList() == 0:
            if pol == 'back':
                    self._serial_num = c_char_p(b"27501898")
                    self._pol = 'back'
            elif pol == 'front':
                self._serial_num = c_char_p(b"27504078")
                self._pol = 'front'
            else:
                print('illegal polariser')
                return

            lib.CC_Open(self._serial_num)
            lib.CC_SetMotorParamsExt(self._serial_num, STEPS_PER_REV,
                     gbox_ratio, pitch)
            lib.CC_Close(self._serial_num)



    def move(self, new_pos_real):
        if lib.TLI_BuildDeviceList() == 0:

            lib.CC_Open(self._serial_num)
            time.sleep(1)
            lib.CC_StartPolling(self._serial_num, c_int(200)) # i dont know what this does
            #set parameter
            lib.CC_SetMotorParamsExt(self._serial_num, STEPS_PER_REV,
                                     gbox_ratio, pitch)

#             # Get the device's current position in dev units
#             lib.CC_RequestPosition(self._serial_num)
#             time.sleep(0.25)
#             dev_pos = c_int(lib.CC_GetPosition(self._serial_num))

#             # Convert device units to real units
#             real_pos = c_double()
#             lib.CC_GetRealValueFromDeviceUnit(self._serial_num,
#                                               dev_pos,
#                                               byref(real_pos),
#                                               0)


            # set a new position in device units
            new_pos_dev = c_int()
            lib.CC_GetDeviceUnitFromRealValue(self._serial_num,
                                              new_pos_real,
                                              byref(new_pos_dev),
                                              0)


            # Move to new position as an absolute move.
            lib.CC_SetMoveAbsolutePosition(self._serial_num, new_pos_dev)
            time.sleep(0.25)
            lib.CC_MoveAbsolute(self._serial_num)
            time.sleep(1)

            # Close the device
#             lib.CC_Close(self._serial_num)
        else:
            print("ib.TLI_BuildDeviceList() is NOT 0")

    def close(self):
        lib.CC_Close(self._serial_num)

    def home(self):
        if lib.TLI_BuildDeviceList() == 0:
            lib.CC_Open(self._serial_num)
            lib.CC_Home(self._serial_num)
            time.sleep(1)
            lib.CC_Close(self._serial_num)

    def get_pos(self):
        if lib.TLI_BuildDeviceList() == 0:
            #open
            lib.CC_Open(self._serial_num)

            #get position in dev units
            lib.CC_RequestPosition(self._serial_num)
            time.sleep(0.25)
            dev_pos = c_int(lib.CC_GetPosition(self._serial_num))
            # Convert device units to real units
            real_pos = float(dev_pos.value)/1919.64186
#             lib.CC_GetRealValueFromDeviceUnit(self._serial_num,
#                                               dev_pos,
#                                               byref(real_pos),
#                                               0)
            lib.CC_Close(self._serial_num)
            return real_pos
        else:
            print("ib.TLI_BuildDeviceList() is NOT 0")



pol = polarizer()

## Play around with motors

In [ ]:
class StopExecution(Exception):
    pass

def stop_execution():
    raise StopExecution()

In [ ]:
def halted_move(pos):
    output = pol.get_pos()
    pol.move(new_pos_real = c_double(pos))
    dif = abs(output-pos)
    time.sleep(4 + dif * 0.2)
    margin = 1
    i = 0

    while i <= 10:
        if (pos - margin) < output and (pos + margin) > output:
            print("Machine output reached the desired value:", output)
            pol.close()
            break  # Exit the loop if the condition is met
        else:
            print("Current output:", output)
            print("Try ", (i + 2), "! Current output:", output, "Retrying...")
            if i == 7 or i == 9:
                pol.home
                time.sleep(5)
            pol.move(new_pos_real = c_double(pos))
            time.sleep(3 + dif/5)
            i = i+1
            if i == 11:
                pol.close()
                print("movement FAILED!!")
                stop_execution()
        output = pol.get_pos()
        dif = abs(output-pos)

# GUI

this GUi should connect to the osciloscope object and control it

In [ ]:
timebase = {('None', '0'),
            ('1ns', '.000000001'), ('2ns', '.000000002'), ('5ns', '.000000005'),
         ('10ns','.00000001'), ('20ns', '.00000002'),  ('50ns', '.00000005'),
         ('100ns','.0000001'), ('200ns', '.0000002'),  ('500ns', '.0000005'),
         ('1µs', '.000001'), ('2µs', '.000002'), ('5µs', '.000005'),
         ('10µs', '.00001'), ('20µs', '.00002'), ('50µs', '.00005'),
         ('100µs', '.0001'), ('200µs', '.0002'), ('500µs', '.0005'),
         ('1ms', '.001'), ('2ms', '.002'),  ('5ms', '.005'),
         ('10ms', '.01'), ('20ms', '.02'),  ('50ms', '.05'),
         ('100ms', '.1'), ('200ms', '.2'),  ('500ms', '.5'),
        ('1s', '1'), ('2s', '2'),  ('5s', '5'),
        ('10s', '10'), ('20s', '20'),  ('50s', '50')}

voff1_write = widgets.FloatText(description = '1 V-Off')

voff2_write = widgets.FloatText(description = '2 V-Off')

chan1_pull = widgets.FloatLogSlider(
    value = scope_cntl._chan1_scale,
    min = -2.69897, max = 1,
    description = 'Channel 1 V')

chan1_write = widgets.FloatText()
widgets.jslink((chan1_pull, 'value'), (chan1_write, 'value'))

chan2_pull = widgets.FloatLogSlider(
    value = scope_cntl._chan2_scale,
    min = -2.69897, max = 1,
    #if channel scale is changed the min and max values should also be chnaged, but thats a tomorrow sam problem
    description = 'Channel 2 V')

chan2_write = widgets.FloatText()
widgets.jslink((chan2_pull, 'value'), (chan2_write, 'value'))

ts_pull = widgets.Dropdown(
    options=timebase,
    description = 'Timescale')

toff_pull = widgets.FloatText(
    description = 'Time Offset',
    value = 0)

toff_scale = widgets.Dropdown(
    options = [('s', 1), ('ms',0.001), ('us', 0.000001), ('ns', 0.000000001)],
    value = 0.001)


powerbutton = widgets.ToggleButtons(
    options=['Start', 'Stop'],
    description='Oscilloscope',
    disabled=False,
)


lvl_pull = widgets.FloatSlider(
    value = 3,
    #for now:
    min = - scope_cntl._trigger_range, max=scope_cntl._trigger_range,
    description = 'Trigger level')
lvl_write = widgets.FloatText()
lvl_link = widgets.jslink((lvl_pull, 'value'), (lvl_write, 'value'))

trig_mode = widgets.Dropdown(
    #these options need to be remade
    options=scope_cntl.modes,
    value='EDGE',
    description='Trigger Mode:'
)

sweep_mode = widgets.Dropdown(
    options = scope_cntl.modes_sweep[trig_mode.value],
    value = 'SING',
    description='Trigger Setup:'
)


trig_sour = widgets.Dropdown(
    options=[('Channel 1', 'CHAN1'), ('Channel 2', 'CHAN2'), ('External', 'EXT')],
    value='EXT',
    description='Trigger Source:',
)

wav_mode = widgets.Dropdown(
    options={('Normal', 'NORM'),('Raw' , 'RAW'), ('Max (Don\'t)', 'MAX')},
    value='RAW',
    description='Waveform Mode:'
)

savername = widgets.Text(description = 'Experiment name',
                             value = 'graphene_exp')

collecter = widgets.Button(description = 'Start Experiment')

# Controls

In [ ]:
def data_test():
    data1, data2 = collect(4, True, True)

    fig, ax = plt.subplots()
    ax.plot(np.linspace(0, 1, num = data1.size), data1, color = 'red', label = 'data 1')
    print("data1: " + str(np.mean(data1)))
    print("data2: " + str(np.mean(data2)))
    data1 = moving_average(data1, 30)
    ax.plot(np.linspace(0, 1, num = data2.size), data2, color = 'blue', label = 'data 2')
    ax.legend()
    fig.show()

In [ ]:
def moving_average(x, w):
    return np.convolve(x, np.ones(w), "valid") / w

In [ ]:
tst_var = ts_pull.value

def start_stop(change):
    if powerbutton.value == 'Start':
        scope_cntl.run
    elif powerbutton.value == 'Stop':
        scope_cntl.stop

def update_timescale(change):
    scope_cntl.set_timescale(timescale = change.new)

def update_timeoffset(change):
    toff_value = '{:.9f}'.format(toff_pull.value * toff_scale.value)
    scope_cntl.set_timescale(offset = str(toff_value))

def update_chan1(change):
    scope_cntl.set_vscale(chan1 = chan1_write.value)

def update_voff1(change):
    scope_cntl.set_voff(chan1 = change.new)

def update_voff2(change):
    scope_cntl.set_voff(chan2 = change.new)

def update_chan2(change):
    scope_cntl.set_vscale(chan2 = chan2_write.value)

def update_trigmode(change):
    scope_cntl.set_trigger(mode = trig_mode.value)
    sweep_mode.options = scope_cntl.modes_sweep[change.new]

def update_trigsour(change):
    scope_cntl.set_trigger(source = trig_sour.value)

def update_triglvl(change):
    scope_cntl.set_triggerlvl(lvl_pull.value)

def update_sweep(change):
    if change.new is not None:
        scope_cntl.set_sweep(change.new)

def update_wav(change):
    scope_cntl.set_wav(change.new)

def collect_save(change):
#     volts, data1, data2 = collect()
#     saver(volts, data1, data2, savername.value)
    data_test()

def change_name(change):
    fname = change.new

powerbutton.observe(start_stop, names = 'value')
ts_pull.observe(update_timescale, names = 'value')
toff_pull.observe(update_timeoffset, names='value')
chan1_write.observe(update_chan1, names='value')
chan2_write.observe(update_chan2, names='value')
lvl_pull.observe(update_triglvl, names ='value')
trig_mode.observe(update_trigmode, names='value')
trig_sour.observe(update_trigsour, names='value')
sweep_mode.observe(update_sweep, names = 'value')
wav_mode.observe(update_wav, names='value')
collecter.on_click(collect_save)
# savername.observe(change_name, names='value')
voff1_write.observe(update_voff1, names='value')
voff2_write.observe(update_voff2, names='value')

### Polarisers control

In [ ]:
setdeg_back = widgets.FloatText(description = 'Analyzer')
setdeg_front = widgets.FloatText(description = 'Halfwave')

def update_front(change):
    pol.set_position(pol = 'front')
    pol.move(new_pos_real = c_double(setdeg_front.value))
    pol.close()
def update_back(change):
    pol.set_position(pol = 'back')
    pol.move(new_pos_real = c_double(setdeg_back.value))
    pol.close()

setdeg_back.observe(update_back, names = 'value')
setdeg_front.observe(update_front, names = 'value')

# GUI is here

In [ ]:
big = HBox([setdeg_back, setdeg_front])


triglvl_box = HBox([lvl_pull, lvl_write, ])
chan_box = HBox([chan1_pull, chan1_write])
chan2_box = HBox([chan2_pull, chan2_write])

trig_box = VBox([trig_mode, trig_sour, sweep_mode])
time_box = VBox([ts_pull, toff_pull, toff_scale])
low_box = HBox([time_box, trig_box])
midlow = HBox([voff1_write, voff2_write])

# deg_box = HBox([pol_setdeg, setdeg_write])
# pol_box = HBox([pol_choice, pol_home])

big_layout = VBox([collecter, chan_box,
                   chan2_box,
                   triglvl_box, low_box, midlow, wav_mode, big])

display(big_layout)